<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Pipeline de Dados do Telegram I
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Contexto;</li>
  <li>Telegram;</li>
  <li>Dados.</li>
</ol>

---

# **Exercícios**

## 1\. Telegram.

Replique as atividades da aula 2.

1.1. Crie uma conta no **Telegram** caso não possua uma e faça o login na versão web da ferramenta através desse [link](https://web.telegram.org).

1.2. Crie um *bot*.

1.3. Crie um grupo e adicione o *bot*.

1.4. Torne o *bot* administrador do grupo.

1.5. Desabilite a opção de adicionar o *bot* a novos grupos.

1.6. Envie diversos tipos de mensagens no grupo (text, imagem, arquivos, video, áudio, etc.) e as consuma utilizand a API de *bots* do **Telegram**.

> **Nota:** A documentação completa da API pode ser encontrada neste [link](https://core.telegram.org/bots/api)

- Criação BOT

Done! Congratulations on your new bot. You will find it at t.me/data_pipeline_ebac_bot. You can now add a description, about section and profile picture for your bot, see /help for a list of commands. By the way, when you've finished creating your cool bot, ping our Bot Support if you want a better username for it. Just make sure the bot is fully operational before you do this.


In [1]:
# injetar a API através do token no python

from getpass import getpass

token = getpass()

··········


In [2]:
# criar URL base

import json

import requests

base_url = f'https://api.telegram.org/bot{token}'

- getME

In [3]:
# retorna informações do BOT

response = requests.get(url=f'{base_url}/getMe')
print(f'{base_url}/getMe')

print(json.dumps(json.loads(response.text), indent=2))

https://api.telegram.org/bot8346440950:AAHLHI74Uj2vsCl6QpP7ky9kX3zg7AD5Mf0/getMe
{
  "ok": true,
  "result": {
    "id": 8346440950,
    "is_bot": true,
    "first_name": "data_pipeline_ebac_bot",
    "username": "data_pipeline_ebac_bot",
    "can_join_groups": false,
    "can_read_all_group_messages": false,
    "supports_inline_queries": false,
    "can_connect_to_business": false,
    "has_main_web_app": false
  }
}


- getUpdates

In [4]:
# retorna as mensagens captadas pelo BOT
response = requests.get(url=f'{base_url}/getUpdates')

print(json.dumps(json.loads(response.text), indent=2))

{
  "ok": true,
  "result": [
    {
      "update_id": 144169603,
      "message": {
        "message_id": 20,
        "from": {
          "id": 8518432228,
          "is_bot": false,
          "first_name": "Nathalia",
          "last_name": "Sa",
          "language_code": "pt-br"
        },
        "chat": {
          "id": -1003415167553,
          "title": "Data Pipeline EBAC",
          "type": "supergroup"
        },
        "date": 1763674352,
        "text": "mensagem para enriqucer"
      }
    },
    {
      "update_id": 144169604,
      "message": {
        "message_id": 21,
        "from": {
          "id": 8518432228,
          "is_bot": false,
          "first_name": "Nathalia",
          "last_name": "Sa",
          "language_code": "pt-br"
        },
        "chat": {
          "id": -1003415167553,
          "title": "Data Pipeline EBAC",
          "type": "supergroup"
        },
        "date": 1763674358,
        "text": "base de dados"
      }
    },
    {
      "u

## 2\. Dados

In [12]:
# escrita em JSON exemplo inicial
%%writefile telegram.json
{
    "update_id": 144169606,
    "message": {
        "message_id": 23,
        "from": {
            "id": 8518432228,
            "is_bot": false,
            "first_name": "Nathalia"
        },
        "chat": {
            "id": -1003415167553,
            "type": "supergroup"
        },
        "date": 1763677343,
        "text": "entre o api e aws"
    }
}


Overwriting telegram.json


### **2.2. Wrangling**

In [14]:
import json

with open('telegram.json', mode='r', encoding='utf8') as fp:
  data = json.load(fp)
  data = data["message"]

In [15]:
print(json.dumps(data, indent=2))

{
  "message_id": 23,
  "from": {
    "id": 8518432228,
    "is_bot": false,
    "first_name": "Nathalia"
  },
  "chat": {
    "id": -1003415167553,
    "type": "supergroup"
  },
  "date": 1763677343,
  "text": "entre o api e aws"
}


-  Denormalizar o conteúdo da mensagem semi-estruturado no formato JSON

In [16]:
# Laço de repetição para varrer todas as chaves do arquivo e selecionar apenas as de interesse
# nesse caso serão apenas mensagens de texto

from datetime import datetime

date = datetime.now().strftime('%Y-%m-%d')
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

parsed_data = dict()

for key, value in data.items():

    if key == 'from':
        for k, v in data[key].items():
            if k in ['id', 'is_bot', 'first_name']:
              parsed_data[f"{key if key == 'chat' else 'user'}_{k}"] = [v]

    elif key == 'chat':
        for k, v in data[key].items():
            if k in ['id', 'type']:
              parsed_data[f"{key if key == 'chat' else 'user'}_{k}"] = [v]

    elif key in ['message_id', 'date', 'text']:
        parsed_data[key] = [value]

if not 'text' in parsed_data.keys():
  parsed_data['text'] = [None]

parsed_data['context_date'] = [date]
parsed_data['context_timestamp'] = [timestamp]

In [17]:
for k, v in parsed_data.items():
  print(f"{k}: {v}")

message_id: [23]
user_id: [8518432228]
user_is_bot: [False]
user_first_name: ['Nathalia']
chat_id: [-1003415167553]
chat_type: ['supergroup']
date: [1763677343]
text: ['entre o api e aws']
context_date: ['2025-11-20']
context_timestamp: ['2025-11-20 22:38:03']


In [18]:
# PyArrow tabela de dados para posteriormente poder salvar em Parquet

import pyarrow as pa

table = pa.Table.from_pydict(mapping=parsed_data)

In [19]:
table

pyarrow.Table
message_id: int64
user_id: int64
user_is_bot: bool
user_first_name: string
chat_id: int64
chat_type: string
date: int64
text: string
context_date: string
context_timestamp: string
----
message_id: [[23]]
user_id: [[8518432228]]
user_is_bot: [[false]]
user_first_name: [["Nathalia"]]
chat_id: [[-1003415167553]]
chat_type: [["supergroup"]]
date: [[1763677343]]
text: [["entre o api e aws"]]
context_date: [["2025-11-20"]]
context_timestamp: [["2025-11-20 22:38:03"]]